In [1]:
!pip install -U transformers sentencepiece bitsandbytes accelerate --quiet

In [2]:
from datasets import load_dataset

ds = load_dataset("squad_v2")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [3]:
trn = ds["train"]
val = ds["validation"]

In [4]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

# Zephyr

Zephyr is a series of language models that are **trained to act as helpful assistants**. Zephyr-7B-α is the first model in the series, and is a fine-tuned version of mistralai/Mistral-7B-v0.1 that was trained on on a mix of publicly available, synthetic datasets using Direct Preference Optimization (DPO). We found that removing the in-built alignment of these datasets boosted performance on MT Bench and made the model more helpful. However, this means that model is likely to generate problematic text when prompted to do so.

In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [36]:
from transformers import Pipeline, TextIteratorStreamer
from threading import Thread

class DecoderPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        # TODO: Prompt Template -> tokenizer.apply_chat_template
        if self.tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        return self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)

    def _forward(self, inputs):
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=200)
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()
        generated_text = ""
        for new_text in streamer:
            generated_text += new_text
        generated_text
        return generated_text

    def postprocess(self, outputs):
        outputs = outputs.replace(".\n", " ")
        outputs = outputs.replace("\n", "")
        return outputs

In [37]:
zephyr = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

In [6]:
prompt = f"""
context:
{trn['context'][0]}
question:
{val["question"][0]}
Please answer the question with information from the given context. 
If you can't answer the question from the given context only answer with 'I don't know.'
Answer:
"""

In [38]:
zephyr(prompt)

'Normandy is located in France, as mentioned in the context that the Normans gave their name to this region in France.'

In [42]:
# clean up
del zephyr
del model
del tokenizer
torch.cuda.empty_cache()

NameError: name 'zephyr' is not defined

# Orca-2-13b

Orca 2 is built for research purposes only and provides a single turn response in tasks such as reasoning over user given data, reading comprehension, math problem solving and text summarization. The model is designed to excel particularly in reasoning.

In [124]:
model_id = "microsoft/Orca-2-13b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

KeyboardInterrupt: 

In [122]:
orca = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

In [123]:
orca(prompt)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

# Mistral-7B-Instruct-v0.1

The Mistral-7B-v0.1 Large Language Model (LLM) is a pretrained generative text model with 7 billion parameters. Mistral-7B-v0.1 outperforms Llama 2 13B on all benchmarks we tested.

In [43]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
mistral = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

In [49]:
mistral(prompt)

NameError: name 'mistral' is not defined

In [48]:
del model
del tokenizer
del mistral
torch.cuda.empty_cache()

# (Intel) neural-chat-7b-v3-1

This model is a fine-tuned model based on mistralai/Mistral-7B-v0.1 on the open source dataset Open-Orca/SlimOrca. Then we align it with DPO algorithm. For more details, you can refer our blog: The Practice of Supervised Fine-tuning and Direct Preference Optimization on Intel Gaudi2.

In [50]:
model_id = "Intel/neural-chat-7b-v3-1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [52]:
intel = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

In [55]:
intel(prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Beyoncé started becoming popular in the late 1990s as lead singer of R&B girl-group Destiny's Child."

In [56]:
del model
del tokenizer
del intel
torch.cuda.empty_cache()

# falcon-7b-instruct

In [57]:
model_id = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
falcon = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

In [63]:
falcon(prompt)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'Normandy is located in France.'

In [64]:
del model
del tokenizer
del falcon
torch.cuda.empty_cache()

# Big Model

In [7]:
from huggingface_hub import login
login()

In [8]:
from torch import bfloat16
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [9]:
from transformers import AutoConfig

model_id = "meta-llama/Llama-2-70b-chat-hf"

model_config = AutoConfig.from_pretrained(
    model_id,
    use_auth_token=True
)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=True
)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
